# Practica 4



El objetivo final de esta práctica es el de implementar la función que permitirá extraer las características para entrenar una red neuronal fully connected.

Veremos a lo largo del cuaderno que extraeremos características a partir del espectrograma lineal y otras a partir del espectrograma mel. 

En la primera parte de la práctica implementaremos algunas de estas características, mientras que otras las extraeremos empleando la librería [`librosa`](https://librosa.org/)

A lo largo de los ejercicios de esta sesión emplearemos el fichero de audio `nine.wav`

## Preparativos

Comenzaremos cargando dicho audio y seleccionando el fragmento de voz tal y como hicimos al final de la practica anterior con la funcion `detect_speech`:

In [ ]:
import librosa
import numpy as np
from utils import detect_speech
import matplotlib.pyplot as plt
from IPython.display import Audio, display # Escuchar audio
import pytest
import UPVlog

In [ ]:
# Read the audio file
nine, fs = librosa.load('nine.wav', sr = None, mono = False)  #mantenemos la frec. original

nine_mask = detect_speech(nine, fs) #guarda aqui el resultado de detect_speech, true cuando haya voz

nine_mask = np.where(nine_mask.astype(bool))[0]

nine_trim = nine[nine_mask] #guarda aqui el audio recortado, que queda solo cn los true de antes, se queda solo cn la voz

t = np.arange(nine.shape[0]) / fs # vector de tiempos para representar nine

t_trim = np.arange(len(nine_trim)) / fs #Vector de tiempos para respresentar nine_trim

In [ ]:
# Comprobación 
my_logger.test("lectura y preprocesado audio")
assert len(nine) == 16000, "lee el ficheo nine.wav"
assert len(nine_trim) == 8880, "Recorta los silencions con detect_speech"
assert t_trim.max() == pytest.approx(0.554, 1e-2)
my_logger.success("lectura y preprocesado audio",2.0)


Representaremos el audio original y recortado:

In [ ]:
fig, axes = plt.subplots(2,1)

axes[0].plot(t, nine)
axes[0].set_title('Nueve original')
axes[0].set_xlabel('time')

axes[1].plot(t_trim, nine_trim)
axes[1].set_title('Nueve')
axes[1].set_xlabel('time')

display(Audio(nine_trim, rate = fs))

En lo que resta de práctica emplearemos como audio de muestra el audio recortado: `nine_trim`


Las primeras características que emplearemos se basan en el espectrograma lineal. Por tanto en primer lugar calcularemos el modulo del espectrograma con los siguientes parámetros:abs
* Longitud de la trama 32ms.
* Solape 50%
* Ventana Hamming
* Numero de puntos de la FFT 512

Use la funciones de `librosa.stft` tal y como hizo en el ejercicio 1 de la práctica 3.


In [ ]:
window = "hamming"
n_fft = 512
tham = 0.032
win_length = int(tham * fs)
hop_length = int(win_length * 0.5)

S = librosa.stft(nine_trim, n_fft=n_fft, hop_length=hop_length, win_length=win_length, window=window) # espectrograma calculado con librosa.stft (esta matriz es compleja)
absS = np.abs(S) # aqui guardaremos el valor absoluto de S

In [ ]:

my_logger.test("generar espectrograma")


assert absS.shape == (257,35), "repasa win_length y hop_length"
assert absS.dtype == np.dtype('float32'), "Toma el valor absoluto de S"
assert np.abs(S[2,2]) == pytest.approx(0.324, abs = 1e-3)


fig, ax = plt.subplots()
librosa.display.specshow(np.abs(S),sr=fs, hop_length=hop_length, x_axis='time', y_axis='linear', ax= ax)
_ = ax.set_title('Espectrograma nine')
my_logger.success("generar espectrograma",2.0)



## Ejercicio 1: Calculo del centroide espectral
Una vez calculado el espectrograma vamos a realizar una función para el cálculo del centroide espectral. El resultado debería ser identico al que obtenemos
 con la función: [`librosa.feature.spectral_centroid`](https://librosa.org/doc/main/generated/librosa.feature.spectral_centroid.html#librosa.feature.spectral_centroid), que utilizaremos para comprobar el resultado.

 La formulá para obtener el espectrograma de la trama `l` es:
 
 $centroid[l] = \frac{\sum_{k=0}^{Nfft / 2} S[k, l] * f[k]}{\sum_{k=0}^{Nfft / 2} S[k, l]}$

 Fíjese que el centroide espectral es un vector de tamaño igual al número de tramas que hay en el espectograma. 

 NOTAS de implementación:

 * Debe programar la función anterior sin realizar ningún bucle, aprovechando el broadcasting de python y la función `np.sum()`
 * La matriz del espectrograma podría tener alguna columna a cero (silencio total durante una trama). En ese caso tendriamos una indeterminación en la ecuación anterior. Este tipo de situaciones se resuelven en la práctica añadiendo un valor insignificante al denominador. Por ejemplo: `eps = 1e-10`. 




In [ ]:
def spectral_centroid(S,f):
    """ Arguments:
        S: Espectrogram (complex matrix obtained using stft)
        f: frequencies vector, vectorf with the corresponding frequency of each row of S
        Returns:
           Vector with the spectral centroid for all frames

        NOTE: use of librosa.feature.spectral_centroid is not allowed
    """
    eps = 1e-10
    S = abs(S) # Tomamos valor absoluto

    numerador = np.sum(S*f[:, None], axis=0)
    denominador = np.sum(S, axis=0) + eps
    
    sc = numerador/denominador # Guarde aqui el vector centroide espectral
    
    return sc

In [ ]:
# Comprobacion 
my_logger.test("spectral centroid")
# Calculamos el vector de frecuencias con librosa
sc_rosa = librosa.feature.spectral_centroid(S = np.abs(S), n_fft=n_fft, sr = fs, window=window)
sc_rosa = sc_rosa.flatten() # librosa devuelve vector fila en lugar de vector 1D

# Calculamos el vector de frecuencias de la fft
f = librosa.core.fft_frequencies(n_fft=n_fft, sr = fs)
# Podiamos obtener alternativamente f de esta forma
# f = np.arange(nn_fft // 2 +1) / nn_fft * fs

sc = spectral_centroid(S, f)

assert f.shape[0] == 257, "dimensiones incorrectas de f"
assert np.abs(sc_rosa - sc).sum() == pytest.approx(0, abs = 1e-2), "resultado no coincide con librosa"


t = np.arange(sc.shape[0])* hop_length / fs # Mira como genero t 

plt.plot(t,sc)
plt.plot(t,sc_rosa.flatten())
plt.xlabel('time')
plt.title('Spectral centroid')
plt.ylabel('Hz')

my_logger.success("spectral centroid",2.0)


## Ejercicio 2: Calculo del  spectral flux

El spectral flux es otra característica que se extrae a partir del espectrograma lineal. 

El resultado es un vector con tantos elementos como tramas tiene el espectrograma. Para calcularlo se emplea la siguiente ecuación:

$flux[l] = \sum_{k=0}^{Nfft / 2} |S[k, l] - S[k, l-1]|^2$

Teniendo en cuenta que:
* flux[0] = 0
* S es el valor absoluto del espectrograma: `S = np.abs(S)`

Teniendo en cuenta esto complete la siguiente función:


In [ ]:
def spectral_flux(S):
    S = np.abs(S) # nos aseguramos de tomar la magnitud
    
    flux = np.zeros(S.shape[1])

    #diferencia de cada trama consecutiva
    diff = S[:, 1:] - S[:, :-1]

    #
    flux[1:] = np.sum(diff**2, axis=0)
    
    return flux

In [ ]:
# comprobacion 
my_logger.test("spectral flux")
sf = spectral_flux(S)

assert sf.shape[0] == 35, "compruebe la función"
assert sf[0] == 0.0, "el primer elemento debe ser cero"
assert sf[10] == pytest.approx(1143.53, 1e-2)


t = np.arange(sc.shape[0])* hop_length / fs # Mira como genero t 

plt.plot(t,sf)
plt.xlabel('time')
plt.title('Spectral flux')
plt.ylabel('Hz')
my_logger.success("spectral flux",2.0)



## Otras características
Las tres funciones siguientes se necesitarán para extraer el vector de características. 

Puede encontrar las fórmulas que se han utilizado en este [enlace](https://es.mathworks.com/help/audio/ug/spectral-descriptors.html)

Fijese que muchas de ellas pueden reutlizar tanto el espectrograma `S` como el vector de frecuencias `f`, además otras funciones pueden reutilizar tanto el `spectral_centroid` si ya lo tiene calculado como el `spectral_spread`.



In [ ]:
def spectral_spread(S, f, centroid = None):
    S = np.abs(S)
    eps = 1e-10
    if centroid is None:
        centroid = spectral_centroid(S,f)

    centroid = centroid.reshape(1,-1)
    f = f.reshape(-1,1)
    
    desv_f = (f - centroid)**2

    spread = np.sqrt((desv_f * S).sum(axis=0) / (S.sum(axis=0)+eps))
    return spread

def spectral_skewness(S,f, centroid = None, spread = None):
    S = np.abs(S)
    eps = 1e-10
    if centroid is None:
        centroid = spectral_centroid(S,f)

    if spread is None:
        spread = spectral_spread(S,f,centroid=centroid)

    centroid = centroid.reshape(1,-1)
    f = f.reshape(-1,1)
 
    desv_f = f - centroid

    num = ((desv_f ** 3)*S).sum(axis=0)
    den = spread**3 * (S.sum(axis=0))
    return num / (den + eps)

def spectral_kurtosis(S,f, centroid = None, spread = None):
    S = np.abs(S)
    eps = 1e-10
    if centroid is None:
        centroid = spectral_centroid(S,f)

    if spread is None:
        spread = spectral_spread(S,f,centroid=centroid)

    centroid = centroid.reshape(1,-1)
    f = f.reshape(-1,1)
 
    desv_f = f - centroid

    num = ((desv_f ** 4)*S).sum(axis=0)
    den = spread**4 * (S.sum(axis=0))
    return num / (den + eps)
    

## Ejercicio 3: Cálculo de los MFCC (Mel Frequency Cepstral Coeficients

Los MFCC se obtienen a partir del espectrograma mel. Los pasos a seguir son los siguientes:
1. Tomar el `log10` del valor absoluto del espectrograma mel
2. Calcular la transformada del coseno de cada una de las columnas de la matriz anterior (el resultado es una matriz). Para ello emplearemos la funcion [`scipy.fft.dct`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fft.dct.html)
3. Quedarnos con las `n_cepstral` filas de la matriz anterior.

Comenzaremos calculando el especrograma mel. En la práctica anterior obteniamos el espectrograma mel a partir de las muestras de la señal. Pero dado que el espectrograma mel se obtiene a partir del espectrograma lineal que hemos empleado para calcular todas las características de los ejercicios anteriores, vamos a obtener el espectrograma mel a partir de la matrix `S` que ya tenemos:


In [ ]:
Smel = librosa.feature.melspectrogram(S= np.abs(S) ,sr=fs,  n_fft=n_fft, 
                                      hop_length=hop_length, 
                                      win_length=win_length,n_mels = 40,
                                      window="hamming") # reutilizamos el espectrograma lineal

In [ ]:
import scipy 

def calculate_mfcc(Smel, n_cepstral):
    eps = 1e-10
    # realice los pasos del enunciado
    #Tomar log10
    log_Smel = np.log10(np.abs(Smel)+ eps) 
    
    #Calcular dct de cada columna (axis = 0)

    mfcc_full = scipy.fft.dct(log_Smel, type=2, axis=0, norm=None)

    #Seleccionar las 13 primeras filas
    mfcc = mfcc_full[:13, :]

    return mfcc 

In [ ]:
# Comprobacion 
my_logger.test("mfcc")
mfcc = calculate_mfcc(Smel, 13)
librosa.display.specshow(mfcc)
assert mfcc.shape == (13,35), "comprueba la funcion" 
assert mfcc[3,3] == pytest.approx(4.4443, abs=1e-3)
my_logger.success("mfcc",2.0)


El resultado obtenido con nuestra función no es exactamente igual al obtenido con la libreria `librosa` por un factor de escala que se produce al tomar el logaritmo. Este factor no es importante en la práctica ya que no cambia la información obtenida. 

Podemos comprobar como descartando dicho factor, el aspecto de MFCC obtenido con `librosa` es idéntico:

In [ ]:
mfcc_rosa = librosa.feature.mfcc(S = librosa.power_to_db(Smel),   n_mfcc=13)
librosa.display.specshow(mfcc_rosa)

En adelante emplearemos la función que proporciona `librosa` para el cálculo de los MFCC. 

## Función para extraer características

A continuación se proporciona el código para extrar las 85 características a partir de un fragmento de voz. 

La función:
1. Quitara silencios iniciales y finales
2. Calculara espectrogramas lineal y mel
3. Extraera las características:
   * mfcc
   * mfcc delta
   * spectral centroid
   * spectral flux
   * spectral spread
   * spectral skewness
   * spectral spectral_kurtosis
   * spectral roll-off



In [ ]:
def extract_features(audio_data, fs):
    win_length = round(0.032*fs)
    hop_length = win_length // 2
    window = 'hamming'
    n_fft = 512
    n_mels = 40
    n_mfcc=13
    f = librosa.core.fft_frequencies(n_fft=n_fft, sr = fs)


    audio_mask = detect_speech(audio_data, fs)
    audio_trim = audio_data[np.where(audio_mask)]
    
    S = librosa.stft(audio_trim, n_fft=n_fft, hop_length=hop_length, win_length=win_length, window=window)
    absS = np.abs(S)

    Smel = librosa.feature.melspectrogram(S= absS ,sr=fs,  n_fft=n_fft, 
                                      hop_length=hop_length, 
                                      win_length=win_length,n_mels = n_mels,
                                      ) # reutilizamos el espectrograma lineal
    
    # Extract MFCCs (Mel-frequency cepstral coefficients)
    mfcc = librosa.feature.mfcc(S = librosa.power_to_db(Smel), sr=fs,  n_mfcc=n_mfcc)

    
    # Calculate delta MFCCs
    
    mfcc_delta = librosa.feature.delta(mfcc)
    
    # Calculate spectral flux
    flux = spectral_flux(absS)

    # Calculate spectral centroid
    centroid = spectral_centroid(absS,f)

    # Calculate spectral spread
    spread = spectral_spread(absS,f,centroid=centroid)

    # Calculate spectral skewness
    skewness = spectral_skewness(absS,f,centroid=centroid, spread=spread)

    # Calculate spectral kurtosis
    kurtosis = spectral_kurtosis(absS,f,centroid=centroid, spread=spread)

    # Calculate spectral rolloff point
    rolloff = librosa.feature.spectral_rolloff(S=absS, sr=fs,  roll_percent=0.85)

    audio_features = {}
    audio_features['mfcc']=mfcc
    audio_features['mfcc_delta']=mfcc_delta
    audio_features['flux']=flux
    audio_features['centroid']=centroid
    audio_features['spread']=spread
    audio_features['skewness']=skewness
    audio_features['kurtosis']=kurtosis
    audio_features['rolloff']=rolloff

    audio_features = vectorize_features(audio_features)
    
    return audio_features


def vectorize_features(audio_features):
    #13 features
    avg_mfcc = np.mean(audio_features['mfcc'],axis=1)
    #13 features
    var_mfcc = np.std(audio_features['mfcc'],axis=1)
    #13 features
    max_mfcc = np.max(audio_features['mfcc'],axis=1)
    #13 features
    min_mfcc = np.min(audio_features['mfcc'],axis=1)
    #13 features
    avg_mfcc_delta = np.mean(audio_features['mfcc_delta'],axis=1)
    #13 features
    var_mfcc_delta = np.std(audio_features['mfcc_delta'],axis=1)

    #2 features
    avg_flux = np.mean(audio_features['flux'])
    var_flux = np.std(audio_features['flux'])

    #5 feature
    avg_centroid = np.mean(audio_features['centroid'])
    avg_spread = np.mean(audio_features['spread'])
    avg_skewness = np.mean(audio_features['skewness'])
    avg_kurtosis = np.mean(audio_features['kurtosis'])
    avg_rolloff = np.mean(audio_features['rolloff'])

                    

    return np.concatenate([avg_mfcc, var_mfcc, 
                           max_mfcc, min_mfcc, 
                           avg_mfcc_delta, var_mfcc_delta,
                           [avg_flux, var_flux],
                           [avg_centroid, avg_spread, avg_skewness, avg_kurtosis, avg_rolloff]])
                           

A continuación vemos como extraer las caracteristicas, a partir de las muestras de un audio:

In [ ]:
features = extract_features(nine,fs)
plt.plot(features)
plt.xlabel('Feature')


En la figura anterior podemos ver el aspecto de las 85 características que emplearemos en la siguiente práctica para clasificar los audios. 

Es importante destacar que el margen dinámico es muy diferente entre las diferentes características, por lo que el primer paso que realizaremos en la siguiente práctica será normalizar estos valores para que se encuentren en rangos dinámicos similares.